In [1]:
# import libraries
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-4
eval_iters = 250

cpu


In [2]:
# import txt file
with open("wizard_of_oz.txt", "r", encoding="utf-8") as f:
    text = f.read()
chars = sorted(set(text))
vocab_size = len(chars)
vocab_size

81

In [3]:
# create encoder and decoder
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l:"".join([int_to_string[i] for i in l])
encoded_hello = encode("hello")
decoded_hello = decode(encoded_hello)
decoded_hello


'hello'

In [4]:
# print first 200 rows
data = torch.tensor(encode(text), dtype=torch.long)
data[:200]

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50,  9,  1, 39, 50, 37, 25,  1, 39,
        30,  1, 39, 50,  9,  1, 29, 44, 27, 11,  0,  0,  1,  1, 33, 36, 36, 45,
        43, 44, 42, 25, 44, 29, 28,  1, 26, 49,  1, 34, 39, 32, 38,  1, 42, 11,
         1, 38, 29, 33, 36, 36,  0,  0,  1,  1, 26, 39, 39, 35, 43,  1, 39, 30,
         1, 47, 39, 38, 28, 29, 42,  1, 47, 33, 36, 36, 33, 25, 37,  1, 37, 39,
        42, 42, 39, 47,  1,  4,  1, 27, 39, 11,  9,  1, 33, 38, 27, 11,  1, 38,
        29, 47])

In [6]:
# split data into training and validation sets 
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

# function to create bacthes
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
print(x.shape)
print(x)
print('targets:')
print(y)


inputs:
torch.Size([4, 8])
tensor([[62, 73, 73, 65, 58,  1, 60, 62],
        [72,  1, 57, 62, 57, 11,  1, 43],
        [58, 59, 68, 71, 58,  1, 61, 58],
        [ 1, 54, 56, 73, 74, 54, 65, 65]])
targets:
tensor([[73, 73, 65, 58,  1, 60, 62, 71],
        [ 1, 57, 62, 57, 11,  1, 43, 68],
        [59, 68, 71, 58,  1, 61, 58,  1],
        [54, 56, 73, 74, 54, 65, 65, 78]])


In [7]:
# loss calculator
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [8]:
import torch.nn as nn
import torch.nn.functional as F

# biagram language model
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape #B=Batch_size, T=Timestamp or number of token in each sentance, c= vocab size also known as channel
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

# context = torch.randint(0, vocab_size, (1, 1))  # (B=1, T=1)


# generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
# len(generated_chars)


.4&XfZB*Dsftri)J3G 8,HaUgvaJkx T&tGMyR2tro&GfWy(WpWs
!?HMEr?j IaqQeq[T28J4zD"LM((6i:j9L!3.
OW1EnqOQ],'9yt8km]1mp"& *V35POdqZAPqkBvJn59GF]&﻿,Cqh3qlv&ie&B'u&rlh[xn.tG'"N26oKgJUaq-HGoIvPb28RK
cF&MP3dLi5PPw59ABAXtbGnVrSkpoJNn0eaAqple8 cUEu&??7 p26kjRRh:G
lwHbcO31S﻿y(D.Z
G u&C4B)tjBvU, '(xXu&]1Jfna_E!8JiF﻿z:D6;-J,Pb,[)tQ'sq6ZKaUIlBj[J9!O3yYTSi2&iXqvLXR9b﻿)g[V"gu&
X9Ve3Gx *)FSaNm
-7guZN
YtbcO.GH9KbmQkuTV[G!mJN&EH?ls?2o*ZMYybOFQys_OVp.
6LX.0X
cUIz?tM(F&
.HRqinrS﻿N;pon"OTW9(BQx8i59GS?sW[,6iL5]0A8Fo5*1JH


In [9]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.890, val loss: 4.869
step: 250, train loss: 4.828, val loss: 4.793
step: 500, train loss: 4.764, val loss: 4.720
step: 750, train loss: 4.713, val loss: 4.683
4.563632965087891


In [10]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


y1ZwJNlzUj'(*DmQ'DSzA?8y; vf3taQ'4
cp28wx
DC4txLGH7p5 F﻿((,e-LVkgo3tL_OH
DX021mNN;FDr;(6-:mnJiD0,dB(O3g'k'zlM21.i'9LDT2YkO&QJ&T*AsI."m:K5K9'E?RSwZ *x 1xjR(7Kn?!yqR:Lo1wD,C&b5M"3TSo1
uQr)wivRDed) RF8VZ,xRB&r o1Jk'Q(4z[w6XWF﻿NWeJf"OTJNfZoexr(65_1tN&ENSu?BAns3MEK]E9Ak!w"wVPbG'04JkK9WRBHjKv)O?O1t6]cPd
OWycU;G3zV(m0'AN9﻿NWELtbIwC;WQz c.h
y1y(?V3SU*A7lM1WFJe"OL
*2&;J.
31_AT[RBCKT*sWhCN7GgyrUZx2F﻿NF8[&DyIMbbXWpdI?s3E!U]v (ZVXguGkz7aC2V[MYn3gNW77 ct[WyS.WmkxY9GS.tvHbbiE:ex (ODLbsA;W5]!f?y_3W8irKKV3e:i6b


In [13]:
import math
print(math.log(81))  # ≈ 4.394


4.394449154672439


model is almost predicting randomly